In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# unscaled, log-normalized counts, with conditions subsampled to the same number of cells 
# and 2000 highly variable genes calculated jointly across all perturbation conditions, including control, using scanpy28 with default parameters (Supplementary Methods)

In [3]:
import scanpy as sc
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import os
from anndata import read_h5ad
import seaborn as sns
import sys
from scipy.spatial.distance import cdist
import networkx as nx
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import pearsonr
sys.path.append("..")
from src import *

will use the CPU to calculate the distance matrix.


/data/bionets/je30bery/singularity/gt/lib/python3.13/site-packages/graph_tool/draw/cairo_draw.py:33: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/data/bionets/je30bery/singularity/gt/lib/python3.13/site-packages/graph_tool/draw/cairo_draw.py:33: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/data/bionets/je30bery/singularity/gt/lib/python3.13/site-packages/graph_tool/draw/cairo_draw.py:33: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/data/bionets/je30bery/singularity/gt/lib/python3.13/site-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo'
  warnings.warn(msg, RuntimeWarning)


In [4]:
# Define a function to categorize the values
def categorize_perturbation(value):
    if value == "control":
        return "control"
    elif "_6" in value:
        return "t_6"
    elif "_24" in value:
        return "t_24"
    else:
        return "unknown"

In [5]:
def jaccard_similarity(matching1, matching2):
    """
    Computes the Jaccard similarity between two graphs given their edge lists.
    """
    edges1 = set(matching1)
    edges2 = set(matching2)
    intersection = len(edges1 & edges2)
    union = len(edges1 | edges2)
    return intersection / union if union != 0 else 0

In [6]:
data_path = "/mnt/data"

In [7]:
os.listdir(data_path)

['sciplex_K562.hdf5',
 'sciplex_A549.hdf5',
 'sciplex_MCF7.hdf5',
 'schiebinger.hdf5',
 'mcfarland.hdf5',
 'norman.hdf5',
 'kang.hdf5',
 'bhattacherjee.hdf5']

In [8]:
adata = read_h5ad(os.path.join(data_path, 'sciplex_A549.hdf5'))

In [9]:
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
adata = adata[:, adata.var.highly_variable]

In [ ]:
sc.pp.neighbors(adata, n_pcs=0)
sc.tl.umap(adata)
sc.pl.umap(adata, color="dose_value")

In [ ]:
asdsaf

In [ ]:
(ref_p, ref_z, ref_s), ref_matching = rosenbaum(adata, group_by="timepoint", reference="control", test_group="t_6", k=None, return_matching=True)

In [ ]:
pd.DataFrame(results, index=["p-val", "z-score", "support", "jaccard similarity"])

In [ ]:
(p, z, s), matching = rosenbaum(adata, group_by="timepoint", reference="control", test_group="t_6", k=2, return_matching=True)

In [ ]:
used_elements = list(chain.from_iterable(matching))

In [ ]:
subset = adata[adata.obs["timepoint"].isin(["control", "t_6"])]

In [ ]:
distances = cdist(subset.X.todense(), subset.X.todense(), metric="sqeuclidean")
lof = LocalOutlierFactor(n_neighbors=20, algorithm='auto', leaf_size=30, metric='precomputed')
pred = lof.fit_predict(distances)
nof = lof.negative_outlier_factor_
outliers = pd.DataFrame(nof, columns=["Negative outlier factor"])
outliers["in matching"] = False
outliers.iloc[used_elements, 1] = True
print(outliers["in matching"].value_counts())

In [ ]:
sns.kdeplot(outliers, x="Negative outlier factor", hue="in matching")
sns.rugplot(outliers, x="Negative outlier factor", hue="in matching")